In [2]:
import requests
import csv
import lxml.html as lh
from bs4 import BeautifulSoup
import pandas as pd
from arcgis.gis import GIS
import urllib
from geopy.geocoders import Nominatim
import folium # map rendering library
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as plt
import numpy as np
from pandas.io.json import json_normalize
import wget
from sklearn.preprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'folium'

Using Beautiful Soup's library functions to extract Postal codes, Borough and Neighbourhood data from Wikipedia page

In [10]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)
html = BeautifulSoup(page.text, 'html.parser')
table = html.findAll('table',{"class":"sortable"})[0]
end_col = len(table.findAll('th'))
end_row = len(table.findAll('tr'))

Logic to extract data with html tags & appending them to empty list

In [11]:
row_data=[]
start_col = 0
start_row = 1   
tr = table.findAll(['tr'])[start_row:end_row]
th = table.find_all(['th'])[start_col:end_col]
th_data = [col.text.strip('\n') for col in th]
for cell in tr:
    td = cell.find_all('td')
    row = [i.text.replace('\n','') for i in td]
    if row[1] != "Not assigned":
        row_data.append(row)

Creating dataframe and setting the columns appropriately

In [12]:
df_toronto = pd.DataFrame(row_data)
df_toronto.columns = th_data
df_toronto.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_toronto

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Using the URL provided in the assignment to get the co-ordinates based on the Postal Code.

In [5]:
df_coordinates = pd.read_csv('http://cocl.us/Geospatial_data')
df_coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476



Merging the coordinates dataframe with the Neighbourhoods dataframe

In [13]:
df_toronto = pd.merge(df_toronto, df_coordinates, on='PostalCode', how='left')
df_toronto

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937



Filtering the data from all the neighbourhoods to with Boroughs containing Toronto in it's name

In [14]:
toronto_data = df_toronto[df_toronto['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


Create a map of Toronto with neighborhoods superimposed on top

In [15]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_data['Latitude'], toronto_data['Longitude'],toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(borough, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

NameError: name 'folium' is not defined

Get & Set Foursquare credentials

In [16]:
CLIENT_ID = '5CMJHFWSA1RHJGPGHBV3NM15NR4J1Z0ACEGYVHIGLLY1KCH5' # your Foursquare ID
CLIENT_SECRET = 'YHUIOAL1GG10GR1C3LBBXFVTI1FBRWWCXL2JO3BPTZIAROP1' # your Foursquare Secret
VERSION = '20200731' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5CMJHFWSA1RHJGPGHBV3NM15NR4J1Z0ACEGYVHIGLLY1KCH5
CLIENT_SECRET:YHUIOAL1GG10GR1C3LBBXFVTI1FBRWWCXL2JO3BPTZIAROP1


In [17]:
toronto_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [18]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude']
neighborhood_longitude = toronto_data.loc[0, 'Longitude']
neighborhood_name = toronto_data.loc[0, 'Neighbourhood']
print(neighborhood_latitude, neighborhood_longitude, neighborhood_name)

43.6542599 -79.3606359 Regent Park, Harbourfront


Get the Four square API URL with relevant details like, longitude, latitude, client ID & Secret

In [19]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=5CMJHFWSA1RHJGPGHBV3NM15NR4J1Z0ACEGYVHIGLLY1KCH5&client_secret=YHUIOAL1GG10GR1C3LBBXFVTI1FBRWWCXL2JO3BPTZIAROP1&v=20200731&ll=43.6542599,-79.3606359&radius=500&limit=100'

Get the results in JSON format

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f29292ee9a64d4c6ee4aac1'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53b8466a498e83df908c3f21',
       'name': 'Tandem Coffee',
       'location': {'address': '368 King St E',
        'crossStreet': 'at Trinity St',
        'lat': 43.65355870959944,
        'lng': -79.36180945913513,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65355870959944,
          'lng': -79.36180945913513}],
        'distance': 122,
       

In [28]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Get Venues and categories of venues with the co-ordinates

In [30]:
venues = results['response']['groups'][0]['items']  
nearby_venues = pd.json_normalize(venues) 
# flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

AttributeError: module 'pandas' has no attribute 'json_normalize'

In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

NameError: name 'nearby_venues' is not defined

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )
toronto_venues

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Sun Life,43.754760,-79.332783,Construction & Landscaping
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
6,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
7,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
8,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
9,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


In [33]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Sun Life,43.754760,-79.332783,Construction & Landscaping
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


Groupby dataframe by neighborhood

In [34]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",27,27,27,27,27,27
Berczy Park,58,58,58,58,58,58
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17


In [35]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


One hot coding toe get the each venue type's occurence.

In [36]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
toronto_onehot.shape

(2162, 272)

In [38]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.037037,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.037037
5,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,"Business reply mail Processing Centre, South C...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.000000,0.066667,0.066667,0.133333,0.2,0.066667,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [39]:
toronto_grouped.shape

(96, 272)

Get top 10 venues

In [41]:
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                             venue  freq
0                   Clothing Store   0.2
1                           Lounge   0.2
2                     Skating Rink   0.2
3                   Breakfast Spot   0.2
4        Latin American Restaurant   0.2
5               Mexican Restaurant   0.0
6  Molecular Gastronomy Restaurant   0.0
7       Modern European Restaurant   0.0
8                Mobile Phone Shop   0.0
9               Miscellaneous Shop   0.0


----Alderwood, Long Branch----
                      venue  freq
0               Pizza Place  0.25
1                  Pharmacy  0.12
2                       Pub  0.12
3            Sandwich Place  0.12
4               Coffee Shop  0.12
5              Skating Rink  0.12
6                       Gym  0.12
7            Medical Center  0.00
8  Mediterranean Restaurant  0.00
9               Men's Store  0.00


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank  0

                  venue  freq
0  Gym / Fitness Center  0.14
1                 Hotel  0.14
2      Department Store  0.14
3        Sandwich Place  0.14
4        Breakfast Spot  0.14
5     Food & Drink Shop  0.14
6                  Park  0.14
7          Hockey Arena  0.00
8                 Motel  0.00
9         Historic Site  0.00


----Del Ray, Mount Dennis, Keelsdale and Silverthorn----
                             venue  freq
0                       Restaurant   0.2
1                   Discount Store   0.2
2                Convenience Store   0.2
3                   Sandwich Place   0.2
4                      Coffee Shop   0.2
5               Miscellaneous Shop   0.0
6              Monument / Landmark   0.0
7  Molecular Gastronomy Restaurant   0.0
8       Modern European Restaurant   0.0
9                Mobile Phone Shop   0.0


----Don Mills----
                 venue  freq
0                  Gym  0.11
1          Coffee Shop  0.07
2                 Café  0.07
3           Beer Store  

                           venue  freq
0                           Café  0.08
1                    Coffee Shop  0.06
2  Vegetarian / Vegan Restaurant  0.06
3          Vietnamese Restaurant  0.05
4             Mexican Restaurant  0.05
5                            Bar  0.05
6                   Burger Joint  0.03
7                         Bakery  0.03
8                   Dessert Shop  0.03
9                           Park  0.03


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                        venue  freq
0                        Park  0.25
1                 Pizza Place  0.25
2                    Bus Line  0.25
3              Sandwich Place  0.25
4          Mexican Restaurant  0.00
5  Modern European Restaurant  0.00
6           Mobile Phone Shop  0.00
7          Miscellaneous Shop  0.00
8   Middle Eastern Restaurant  0.00
9               Metro Station  0.00


----Lawrence Manor, Lawrence Heights----
                    venue  freq
0          Clothin

              venue  freq
0       Coffee Shop  0.09
1              Café  0.05
2    Clothing Store  0.04
3        Restaurant  0.04
4             Hotel  0.04
5               Gym  0.04
6   Thai Restaurant  0.03
7               Bar  0.03
8        Steakhouse  0.03
9  Sushi Restaurant  0.02


----Rosedale----
                             venue  freq
0                             Park  0.50
1                            Trail  0.25
2                       Playground  0.25
3  Molecular Gastronomy Restaurant  0.00
4       Modern European Restaurant  0.00
5                Mobile Phone Shop  0.00
6               Miscellaneous Shop  0.00
7        Middle Eastern Restaurant  0.00
8               Mexican Restaurant  0.00
9                    Metro Station  0.00


----Roselawn----
                             venue  freq
0                     Home Service   0.5
1                           Garden   0.5
2              Moroccan Restaurant   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern Europ

                             venue  freq
0                             Park   0.5
1                Convenience Store   0.5
2                      Yoga Studio   0.0
3                    Metro Station   0.0
4  Molecular Gastronomy Restaurant   0.0
5       Modern European Restaurant   0.0
6                Mobile Phone Shop   0.0
7               Miscellaneous Shop   0.0
8        Middle Eastern Restaurant   0.0
9               Mexican Restaurant   0.0


----Wexford, Maryvale----
                             venue  freq
0                           Bakery  0.17
1        Middle Eastern Restaurant  0.17
2                   Breakfast Spot  0.17
3                      Auto Garage  0.17
4            Vietnamese Restaurant  0.17
5                   Sandwich Place  0.17
6  Molecular Gastronomy Restaurant  0.00
7       Modern European Restaurant  0.00
8                Mobile Phone Shop  0.00
9               Miscellaneous Shop  0.00


----Willowdale, Newtonbrook----
                             venue  

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Get top 100 venues

In [52]:
num_top_venues = 100

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :],num_top_venues)

neighborhoods_venues_sorted.head()

NameError: name 'np' is not defined

In [44]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:200]

NameError: name 'KMeans' is not defined

In [47]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

neighborhoods_venues_sorted

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged # check the last columns!

NameError: name 'neighborhoods_venues_sorted' is not defined

Clusters displayed with venues in each of the neighbourhoods

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

NameError: name 'toronto_merged' is not defined

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

NameError: name 'toronto_merged' is not defined